In [1]:
from fake_useragent import UserAgent
from os.path import join,isfile
from os import listdir
import dask
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from collections import Counter
import json
import re
import time
import bookquery
import sys
import os
import datetime
import math
import numpy as np

ua = UserAgent()#Прокси
curr_directory = os.getcwd()
new_path = join(curr_directory,"Information")
try:# Создаем базовую директорию для файлов
    os.mkdir(new_path)
except OSError:
    print ("Директория %s уже создана" % new_path)
    
def execute_query(qquery, entity_id):
    
    """Работа с запросами"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
    
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

Директория C:\Users\Алексей Кушнер\Sberbank\Geography\Information уже создана


In [2]:
def entities_sparqul(results):
    
    """Обработка запроса"""
    
        res = []
        for result in results:# Обработка вернувшегося запрсов
            entities = {}
            entities['entity'] = result['item']['value'].split('/')[-1]
            entities['label'] = result['itemLabel']['value']
        return entities#Возвращаем Датафрейм и название сущности
    
def processing(entity_id,label, f = 0 ):
    
    """Обработка полученного запроса"""
    
    if f>5:
        print(entity_id)
        errors +=1
        resu = {'entity' : entity_id , 'label' : label}
        return resu
    try:
        raw_results= execute_query(bookquery.rus_names, entity_id)# Запрос
        resu = entities_sparqul(raw_results)
        return resu
    except Exception as e:
        time.sleep(3)
        return processing(entity_id,label, f+1)
        
def file_creation(file,path):
    
    """Формирование файла"""
    
        file_name = file.split('\\')[-1]
        start = time.time()
        new_path_2 = join(os.getcwd(),join("Information",path + "\\data\\rus_labels"))
        if file_name not in list (map(lambda x:x.split('\\')[-1],counter_for_files(new_path_2))):#Проверка на наличие файла
            print(f'Файл {file_name } начал обрабатываться {datetime.datetime.now().time()}')
            df = pd.read_csv(file ,encoding='utf8',sep = ';')
            result = []
            errors = 0
            for k, v in df.iterrows():#Запросы по каждой сущности
                result.append(dask.delayed(processing)(v['entity'] , v['label']))
            data = list(dask.compute(*result))
            results = pd.DataFrame(data = data)
            results.to_csv(join(new_path_2 ,file_name),
                          encoding='utf-8-sig',index = False,sep = ';')# Формирование
            end = time.time()
            print(f'Формирование файла {file_name} окончено , заняло {(end-start)/60} минут\n , ошибок {errors/len(df)*100}')
        else :
            print(f'Файл {file_name} уже обработана')  
        
def main_part():
    
    """Основная функция для прохождения по всем файлам директории"""
    curr_path =join(os.getcwd(),join("Information","2" + "\\data"))
    for file in counter_for_files(curr_path):
        file_creation(file , '2')
        
main_part()

Файл entities_aerodrome_Q62447.csv уже обработана
Файл entities_airport_Q1248784.csv уже обработана
Файл entities_archipelago_Q33837.csv уже обработана
Файл entities_autonomous oblast of Russia_Q309166.csv уже обработана
Файл entities_autonomous okrug of Russia_Q184122.csv уже обработана
Файл entities_bay_Q39594.csv уже обработана
Файл entities_bridge_Q12280.csv уже обработана
Файл entities_canal_Q12284.csv уже обработана
Файл entities_canyon_Q150784.csv уже обработана
Файл entities_cape_Q185113.csv уже обработана
Файл entities_cave_Q35509.csv уже обработана
Файл entities_channel_Q1210950.csv уже обработана
Файл entities_city of the United States_Q1093829.csv уже обработана
Файл entities_city town_Q7930989.csv уже обработана
Файл entities_city_Q515.csv уже обработана
Файл entities_constituent part of the United Kingdom_Q3336843.csv уже обработана
Файл entities_continental area and surrounding islands_Q2418896.csv уже обработана
Файл entities_continent_Q5107.csv уже обработана
Файл enti

KeyboardInterrupt: 

In [4]:
def entities_sparqul(results):
    
    """Обработка запроса"""
    
        res = []
        for result in results:# Обработка вернувшегося запрсов
            entities = {}
            entities['entity'] = result['item']['value'].split('/')[-1]
            entities['label'] = result['itemLabel']['value']
        return entities#Возвращаем Датафрейм и название сущности
    
def processing(entity_id,label, f = 0 ):
    
    """Обработка полученного запроса"""
    
    if f>5:
        errors +=1
        resu = {'entity' : entity_id , 'label' : label}
        return resu
    try:
        raw_results= execute_query(bookquery.rus_names, entity_id)# Запрос
        resu = entities_sparqul(raw_results)
        return resu
    except Exception as e:
        time.sleep(3)
        return processing(entity_id,label, f+1)
        
def creating_parts(i, ids, file_name,path):
    
    """Формирование блоков по 50000 представителей"""
    
    new_path = join(curr_directory,join("Information",path +"\\data\\big\\parts"))
    if str(i) not in list(map (lambda x : x.split('_')[-1].split('.')[0],counter_for_files(new_path))):
        print(f'Файл {file_name}_{i} начал обрабатываться {datetime.datetime.now().time()}')
        start = time.time()
        res = []
        result = []   
        for k, v in ids.iterrows():#Запросы по каждой сущности
            result.append(dask.delayed(processing)(v['entity'] , v['label']))
        data = list(dask.compute(*result))
        results = pd.DataFrame(data = data)
        results.to_csv(join(new_path ,f'{file_name}_{i}.csv'),
                          encoding='utf-8-sig',index = False,sep = ';')# Формирование
        end = time.time()
        print(f'Формирование файла {file_name}_{i} окончено , заняло {(end-start)/60} минут\n')
    else:
        print(f'{i} файл уже обработан')
        
        
def file_creation(file,path):#Создание файла
    
     """Создает файлов с блоками по  50000 представителей """
        
        BLOCK_SIZE = 50000
        file_name = file.split('\\')[-1].split('.')[0]
        new_path_1 = join(os.getcwd(),join("Information",path + "\\data\\big\\parts\\result"))
        if file_name not in list (map(lambda x:x.split('\\')[-1].split('.')[0],counter_for_files(new_path_1))):#Проверка на наличие файла
            start = time.time()
            print(f'Файл {file_name } начал обрабатываться {datetime.datetime.now().time()}')
            ids = pd.read_csv(file ,encoding='utf8',sep = ';')
            for i in range(math.ceil(len(ids)/BLOCK_SIZE)):
                if (i+1) *BLOCK_SIZE< len(ids):
                    creating_parts(i , ids[BLOCK_SIZE * i: BLOCK_SIZE*(i+1)] ,file_name,path)
                else:
                    creating_parts(i , ids[BLOCK_SIZE * i:],file_name,path)
            end = time.time()
            print(f'Формирование файла {file_name} окончено , заняло {(end-start)/60} минут\n')
        else :
            print(f'Файл {file_name} уже обработана')  
        
def main_part():
    curr_path =join(os.getcwd(),join("Information","2" + "\\data\\big"))
    resu = []
    for file in counter_for_files(curr_path):
        file_creation(file , '2')
        
main_part()

Файл entities_settlement_Q532 начал обрабатываться 09:27:44.444520
Файл entities_settlement_Q532_0 начал обрабатываться 09:27:44.778172
Формирование файла entities_settlement_Q532_0 окончено , заняло 16.17502841949463 минут

Файл entities_settlement_Q532_1 начал обрабатываться 09:43:55.342381
Формирование файла entities_settlement_Q532_1 окончено , заняло 17.55528246164322 минут

Файл entities_settlement_Q532_2 начал обрабатываться 10:01:28.706206
Формирование файла entities_settlement_Q532_2 окончено , заняло 17.446495532989502 минут

Файл entities_settlement_Q532_3 начал обрабатываться 10:18:55.558444
Формирование файла entities_settlement_Q532_3 окончено , заняло 17.318514422575632 минут

Файл entities_settlement_Q532_4 начал обрабатываться 10:36:14.747434
Формирование файла entities_settlement_Q532_4 окончено , заняло 17.296521544456482 минут

Файл entities_settlement_Q532_5 начал обрабатываться 10:53:32.601224
Формирование файла entities_settlement_Q532_5 окончено , заняло 8.79281

In [6]:
path = join(os.getcwd(),join("Information","2" + "\\data\\big\\parts"))
final_path = join(os.getcwd(),join("Information","2" + "\\data\\big\\parts\\result"))
def concat_sets(path):
    df  =[]
    for i in counter_for_files(path):
        part_df = pd.read_csv(i, encoding='utf8',sep = ';')
        df.append(part_df)
    name = counter_for_files(path)[0].split('\\')[-1].split('.')[0].split('_')[:-1]
    name = '_'.join(name)
    df = pd.concat(df)
    df.to_csv(f'{final_path}\\{name}.csv' , encoding='utf-8-sig',sep = ';' , index = False)
concat_sets(path)    